#Geospatial analysis using python
#Lab 6: Task 2
#Submitted by: Hossain Idrish

In [87]:
# Mounting Google Drive in Google Colab for File Access
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Installing rasterio and geopandas Libraries in Colab
!pip install rasterio geopandas


In [ ]:
#import packages
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio.mask

In [70]:
# Load the GeoJSON file to check its content
geojson_path = '/content/drive/MyDrive/GIS-4120-50/Lab6/task2/grid.geojson'

# Read the GeoJSON file
geojson_data = gpd.read_file(geojson_path)

# Display the first few rows to inspect the content
geojson_data.head()


,id,left,top,right,bottom,row_index,col_index,geometry
0,41,583445.254038,4258815.0,594992.259422,4248815.0,16,1,"POLYGON ((583445.254 4253815, 586332.005 42588..."
1,42,583445.254038,4248815.0,594992.259422,4238815.0,17,1,"POLYGON ((583445.254 4243815, 586332.005 42488..."
2,61,592105.508076,4303815.0,603652.513459,4293815.0,12,2,"POLYGON ((592105.508 4298815, 594992.259 43038..."
3,62,592105.508076,4293815.0,603652.513459,4283815.0,13,2,"POLYGON ((592105.508 4288815, 594992.259 42938..."
4,63,592105.508076,4283815.0,603652.513459,4273815.0,14,2,"POLYGON ((592105.508 4278815, 594992.259 42838..."


In [81]:
def zonal_statistics(raster_path, vector_path, unique_id_column):
    """
    Computes the mean and standard deviation of raster values for each polygon in the vector file.

    Parameters:
        raster_path (str): Path to the single-band raster file (e.g., NDVI).
        vector_path (str): Path to the vector file (e.g., shapefile or GeoJSON).
        unique_id_column (str): Column name in the vector file with unique IDs for polygons.

    Returns:
        pd.DataFrame: DataFrame with unique ID, mean, and standard deviation of raster values.
    """
    # Load the vector data
    vector_data = gpd.read_file(vector_path)

    # Open the raster data
    with rasterio.open(raster_path) as src:
        raster_crs = src.crs

        # Automatically reproject vector data if CRS does not match
        if vector_data.crs != raster_crs:
            print(f"Reprojecting vector data from {vector_data.crs} to {raster_crs}.")
            vector_data = vector_data.to_crs(raster_crs)


        # Initialize a list to store results
        stats = []

        # Loop through each feature in the vector data
        for i, row in vector_data.iterrows():
            geom = [row['geometry']]
            unique_id = row[unique_id_column]

            try:
                # Mask the raster with the polygon geometry
                out_image, _ = rasterio.mask.mask(src, geom, crop=True)
                masked_array = out_image[0]  # Extract the single band

                # Filter out nodata values
                masked_array = masked_array[masked_array != src.nodata]

                # Compute mean and standard deviation
                if masked_array.size > 0:
                    mean_val = masked_array.mean()
                    std_val = masked_array.std()
                else:
                    mean_val, std_val = np.nan, np.nan

            except ValueError as e:
                # Handle polygons that do not intersect the raster
                if "do not overlap" in str(e):
                    print(f"Polygon with ID {unique_id} does not overlap raster. Skipping...")
                    mean_val, std_val = np.nan, np.nan

            # Append results
            stats.append({'unique_id': unique_id, 'mean': mean_val, 'std': std_val})

    # Convert results to a DataFrame
    stats_df = pd.DataFrame(stats)
    return stats_df

In [82]:
# Defining Raster and Vector Paths for Zonal Statistics Computation
raster_path = "/content/drive/MyDrive/GIS-4120-50/Lab6/task2/ndvi.tif"
vector_path = "/content/drive/MyDrive/GIS-4120-50/Lab6/task2/grid.geojson"

In [83]:
#create the dataframe using function
df = zonal_statistics(raster_path,vector_path,'id' )

In [88]:
# Displaying the DataFrame with Zonal Statistics Results
df

,unique_id,mean,std
0,41,0.405855,0.067818
1,42,0.406319,0.058588
2,61,0.424627,0.065736
3,62,0.417399,0.071759
4,63,0.363553,0.140118
...,...,...,...
367,560,0.333018,0.111100
368,561,0.334814,0.114300
369,562,0.301963,0.115530
370,581,0.312016,0.097210
